In [1]:
import sqlite3
import pandas as pd


In [2]:

conn = sqlite3.connect("onet_data.db")


def load_csv_to_sqlite(file_path, table_name, chunk_size=50000):
    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        chunk.to_sql(table_name, conn, if_exists="append", index=False)


load_csv_to_sqlite(r"C:\Users\Skmal\OneDrive\Desktop\datasets\Occupation Data.csv", "occupation_data")
load_csv_to_sqlite(r"C:\Users\Skmal\OneDrive\Desktop\datasets\Skills.csv", "skills_data")
load_csv_to_sqlite(r"C:\Users\Skmal\OneDrive\Desktop\datasets\Task Statements.csv", "task_statements")
load_csv_to_sqlite(r"C:\Users\Skmal\OneDrive\Desktop\datasets\Work Activities.csv", "work_activities")
load_csv_to_sqlite(r"C:\Users\Skmal\OneDrive\Desktop\datasets\Technology Skills.csv", "technology_skills")
load_csv_to_sqlite(r"C:\Users\Skmal\OneDrive\Desktop\datasets\Knowledge.csv", "knowledge_data")


In [3]:

query = """
SELECT o.Title, o.Description, 
       s."Element Name" AS Skills, 
       t.Task, 
       w."Element Name" AS Activities, 
       ts.Example AS TechnologySkills, 
       k."Element Name" AS Knowledge
FROM occupation_data o
LEFT JOIN skills_data s ON o."O*NET-SOC Code" = s."O*NET-SOC Code"
LEFT JOIN task_statements t ON o."O*NET-SOC Code" = t."O*NET-SOC Code"
LEFT JOIN work_activities w ON o."O*NET-SOC Code" = w."O*NET-SOC Code"
LEFT JOIN technology_skills ts ON o."O*NET-SOC Code" = ts."O*NET-SOC Code"
LEFT JOIN knowledge_data k ON o."O*NET-SOC Code" = k."O*NET-SOC Code"
LIMIT 50000
"""



job_profiles = pd.read_sql_query(query, conn)


In [4]:

job_profiles["Job Profile"] = (
    job_profiles["Title"] + " " +
    job_profiles["Description"].fillna("") + " " +
    job_profiles["Skills"].fillna("") + " " +
    job_profiles["Task"].fillna("") + " " +
    job_profiles["Activities"].fillna("") + " " +
    job_profiles["TechnologySkills"].fillna("") + " " +
    job_profiles["Knowledge"].fillna("")
)


job_profiles.to_csv("job_profiles.csv", index=False)
print(f"Saved {len(job_profiles)} job profiles to job_profiles.csv")


Saved 50000 job profiles to job_profiles.csv


In [5]:
from sentence_transformers import InputExample
import pandas as pd


job_profiles = pd.read_csv("job_profiles.csv")


training_examples = []
for _, row in job_profiles.iterrows():
    training_examples.append(InputExample(
        texts=[row["Title"], row["Job Profile"]],
        label=1.0  
    ))


In [ ]:
from sentence_transformers import SentenceTransformer, losses
from torch.utils.data import DataLoader


model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


train_dataloader = DataLoader(training_examples, shuffle=True, batch_size=16)


train_loss = losses.CosineSimilarityLoss(model)


def train_with_checkpointing(model, train_dataloader, train_loss, epochs=3, output_path="job_title_model"):
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        model.fit(
            train_objectives=[(train_dataloader, train_loss)],
            epochs=1,  
            warmup_steps=100,
            output_path=f"{output_path}_epoch_{epoch + 1}"  
        )
        print(f"Checkpoint saved for epoch {epoch + 1} at {output_path}_epoch_{epoch + 1}")


train_with_checkpointing(model, train_dataloader, train_loss, epochs=3, output_path="job_title_model")


In [9]:
from sentence_transformers import SentenceTransformer


model.save("job_title_model_final")
print("Model saved")


Model saved
